In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.genz.file_names import project, Optsd
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Optsd()
style = {'description_width': 'initial'}
input_selkey = widgets.Dropdown(options=sorted(get_h5_keys(opts.info_fname, key='auto_qc')), disabled=False,
    description='Subject DWI to qc:', style=style)
input_selkey.layout.width = '350px'
input_selkey

/home/toddr/Software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


Dropdown(description=u'Subject DWI to qc:', layout=Layout(width=u'350px'), options=('/sub-genz103/ses-1/dwi/auto_qc', '/sub-genz104/ses-1/dwi/auto_qc', '/sub-genz201/ses-1/dwi/auto_qc', '/sub-genz203/ses-1/dwi/auto_qc', '/sub-genz205/ses-1/dwi/auto_qc', '/sub-genz301/ses-1/dwi/auto_qc', '/sub-genz302/ses-1/dwi/auto_qc', '/sub-genz303/ses-1/dwi/auto_qc', '/sub-genz305/ses-1/dwi/auto_qc', '/sub-genz308/ses-1/dwi/auto_qc', '/sub-genz501/ses-1/dwi/auto_qc', '/sub-genz502/ses-1/dwi/auto_qc', '/sub-genz503/ses-1/dwi/auto_qc', '/sub-genz506/ses-1/dwi/auto_qc', '/sub-genz508/ses-1/dwi/auto_qc', '/sub-genz510/ses-1/dwi/auto_qc', '/sub-genz906/ses-1/dwi/auto_qc'), style=DescriptionStyle(description_width=u'initial'), value='/sub-genz103/ses-1/dwi/auto_qc')

In [2]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
# new h52df function makes all cols numeric. yes!
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
# todo: make python scriptlet to load files into FSLeyes
# fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
# with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
#     with open('fslview_cmd.sh', 'w') as f:
#         f.write('fslview ' + fslv_names + ' &\n')

# limit to only active rows with mask
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] != -9999, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] != -9999, 'itopdn_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] == -9999, 'itopup_visqc'] = False
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] == -9999, 'itopdn_visqc'] = False
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)

In [3]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600),]
#items_layout = [widgets.Layout(flex-basis='50%', ), widgets.Layout(flex-basis='50%', widgets.Layout(flex-basis='100%',)]
box_layout = widgets.Layout(display='flex', align_items='stretch', border='solid', width='100%')
top_box = widgets.HBox(children=items)
bottom_box = widgets.HBox(children=[dwi_w])
dwi_box = widgets.Box([top_box, bottom_box], layout=box_layout)
dwi_box

Box(children=(HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xbf\xbf\xbf\x1f\x1f\x1f___???\xdf\xdf\xdf\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xf5Uk\xc9\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\x9d\xd9v\xab:\x16\x00\x9bI?\xc7[\xe6\xe9$\xb9\xc9\x82\xff\xff\x84\xb6F$!l\xe3`l\xe1\xaa\xd5}\x8f\x831[`(o\tI\xfc\xaf\x07\x00\xc8\x84\xff]\xba\x00\x00\x00\xc7\x82\xb0\x00 \x1b\x10\x16\x00d\x03\xc2\x02\x80l@X\x00\x90\r\x08\x0b\x00\xb2\x01a\x01@6 ,\x00\xc8\x86[\x11V\xd7U\x97.\xc2\x85\xe8,UU\x97zQ\xdb\xc5\xd4\xbb\xa5U\xd7\xa5>_\xd4U%?[D\xcb\xcb\xb6\xd9-oF\xcb\x13Lly%\xaa\xe1\x08\xc4\xa5-\xba\xe8\xbcH\x1c\xac\xb9\xb1\x0e\xac!\xea\xea\xe8S\xf1vO\xdai\x10\xd6\x921\x86\xf3\xdd.+\xe5\xf9\xd9\xf8\x97IY7\xf2\xca9\xe5r8\x80H^-\xa1\x98\x84\\4CX\xadw\xb9\xb7\xfe^4\xc3\xf2\x83\xca:\x9b\xb0\x82=\x9e:\xb0U\xb0\xa7\x8d\xf0\xdeR;\xe1\x7f q\xb0fqpO\x85\xda\xf2\x91[K\x9e\xb4\x85\xfe\x9d\x10\xc3\x92\xd49vxI\xa6 \xac\xc5\x10\ta\xd9\xcb\xba\x11v%w\xa1\xbbE\x8bQL\t\xabV\xa8\xc8\x95\x8cZ\xe8\x05\xb5{K%\x1e\x89\x8bM\x97vwy4UX\xe4"X\xde\xc6\x9f\x8b8\x9b\xb0\xbc=\x9e>\xb0\x95\xdbKUZ\xef<\x10N\xd6\x96\xc4\xc1\x9a\xc5\xc1=\xdd\x1d\xf3\xa6=\xd6\x1b\x89\x93\xb6t\xfau\x07}t\x8e\x1d\xb5$W\x10\xd6b\x94\xf2\x126\x98\x8b@\x9e\'u[\x0f\x97\x89\xa8\xf4\x12\xf9F\xb3p|\x91\xbeZ\xbc=/\xaa0j\xf8\x81\xf1\xc7Ui[\xa1_\xb7\xde\x15\\\xca\xe2\xeb\xccD\xa5Z\x07\xd2\xc5s\t\xcb\xdb\xe3=\x07\xd6\x8f.E[\xf8\x7f\xd5\xe1\x89\xb1\xe7`\x1d\xc5\xc1=\xdd\xad \x8e\xde\xda\xf8\xa4-;\xbd\x9b\xd5 \xda\xd19v\xd4\x92lAX\x8bQ\x8c.\xdc\xc2\\\xe2\xf2\xd7_\xff \xd6\xf6GN^`\x87\x12\x93\x99\xe1\xabt]\xc3\xdf\xf3(\xea!a5A\x92!\x86+8\xd8L}\xf0\xc2>\x93\xb0\xfc=\xdes`\x83\xe8\x85\x9fQ\xed\xf6O\xfa\xd6\xfb\xd6\xf6\x1c\xac\xa38FX\xc7om|\xd26v7\x0b\xfb31>\xc7\x8eY\x92/\x08k1\xda\xd1\x8fge/\x06a\xaf\xac\xe1\x07\xb6\\4\xc5\xaaMM!\xf1V\xb0\xe7e\xf0\xd7\x01a\x15Q\xa5H^\xc1*=iG\xf5(\x7f\xb51\xe7\x10V\xb4\xc7{\x0el\x10]x\xef\ny0\x02\x83\xed;XGqfay%\xb2\xdfA\xf2\x1c;\xb8$_6#,\xd3\x16i~-\xbd\xafZ\xbf\x94\xff\x15\xb2\x05\xb91U\x9c\x88V\xfev\xb5\xfe\t\xa57\xe8\xea\x0f\xea\r\xddt\xe9bx\xf4\xeaG>\xdc\xa6\x08r\xf2",\x97\x18\t+\xdc\x83q\t\x12k8\xaa\xee8a\xf5ABq@XU\x9c2\xda\xeb%\xaa\xd7\xd4~5+UR\xb5e\xbd(\xde;\x97x\xea7\n\'\xbf\xd6$\x03S\xfb\x1c\xed\xf1\x9e\x03;),\x1d"8j{\x0e\xd6P$\x7f\x9f\xfd\xbd\xf0c\xa5\x8b\xed\x9d-\xc9O\xeer\xc6"X=:i\xbd\x12\x98\x93k|\x8e\x1d\xb3$\xc0D\xd8s\xee_\x13\x1b\x11\xd6\xd0\x16\xa9\x7f.S\xc2\xb2\xebT\xe3\x0b\xde\xbd\xe5ZE\x84\xdd\xa0\xcd1\xe4\x1bm\x14#\x14V\x13\xff\x1e\x16\xc3\xe9e_\x0e\x0e\x08\x7f\xd9\xc7{0.\xc1h\r\xff\xd3\xc5\x8e\x83mXa\x99\x0e\t+\x91\x03\xea+\xb8\x8cJ \xcaR\x84\x9f\x8cJ*\xb7\xdcL\xed]\xe9\x97J6\xb3\x14~\xa8\xc9}\x8e\xf6x\xfa\xc0\x86\xfb\xd5zW\xac\xf6n\xa0\xdb=\x07+\xb8\xf0\x8d\xe6\xc2\xbd\x18bM\x15{8[\x92\x9fl\xbb.\x16Vx\xd2\xd6\x9eAu\xa8\xf19v\xcc\x12\x9f}\xe7\xfe\x19ne\xff\x95m\x08K\x1e\xe2\xaane\x9a\xa4\xbf\x90\x94\xb0\xd4O\x9e\xbanD\xfcq\xf9

In [6]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xdf\xdf\xdf___???\xbf\xbf\xbf\x1f\x1f\x1f\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xa8*\xe1\x90\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\xdd\t\x96\xa2J\xb7\x06\xd0G\x17\x93c\x18\xff\xc2\xf9\x0f\xe1I\x13H\xa7)U&yO\xd6\xdek\xdd[\xa9"\x07\x10>!\x08\xf0\xffn\x00A\xfc\xdfOO\x00\xc0\xbb\x04\x16\x10\x86\xc0\x02\xc2\x10X@\x18\x02\x0b\x08C`\x01a\x08, \x0c\x81\x05\x84\xf1{\x03\xab\xeb\xd2OO\xc2\x0f\xe9\xb2\x94\xaaf|\xaa\xed\xb6\xaa\xfb\xb3\xa9\xeb\x8e\xde_T)\xf5\xef-6\xcf7m}\x7f\xbe\xde=\x7f\xe0\xc9\x98\xbfIz\xcc\xf1v\xea\x8an\xb3\x1e\x1c,\x9c\xb3\xb5\xbe\x18\xa2\xac\xd2\xba\xe4\x8f\xfam\x9b\x81\xc0\xfa\xbb\x1a\x8f\xf5??\xd7\xf4\xebk\xbd\xdcl\x9a\xaa\xee\xb7\xa4?\xd9<\xbeP\x1en=\xeb`*\xfb\xa7N\x04V\xbb\xd8\xfc\xdb\xe5\\\xd4\x8f\xe7\xbf\x8c\xac\xef\t\xac\xfd\xa2\x9d\xab-\xd4\xe5\xe2\xa5a\xa2\x97K\xfe`\xe1\x9c\xf2\xe5\x9c\x95\xc3\x98\xdf\x1c[[U\xa7\xa7\xe0\x0b\x9b\xb5\xedp3x\xb6 \x03\x10X\x7f\xa1<\x08\xacz\xbb\xd9\x94\xf3\x86^\x9f\xdf<\xbeP<\x0b\xacj0TN}\xd5b|\xa2\x9a_\x1avD\x0e6\xbeqj\xef+|\x9d\xd6\x93\\\xac\x9eo\xb7\xef\xdb\xf8\x96\xc0\xda-\xdaE\xb5i\xae\x86\xa9K\x8fW\xca9\x9c\xb3\x83\x85s\xca\x97sv_\xc6u\xfbn\x16||9\xed\xd6\xb6\xa3\xcd\xe0\xe9\x82\x0c@`\xfd\x85\xa6\xdf\x84\'\xd3F\xd1\xaf\x0bU[=6\x9b2\x8d\xcf\xf4/\xd4\x1f\xae_\x1e\xaf\xef\x8b9/\xd2\xba\xea\xfa\r\xfb\xb7\x0fS\xdb\x8e\xebq\xd9.\xb6\xe8\xa6\x9f\xfc\xf1[{\xd8\xd5\xfabw\xf1;\x02k\xb7h\x0f\xab\xf5\xc1Z,\x1fU\xeb\xe1_,\x9c\xb7|9g\xf7\x01\xde\xcf\x81O/\xa7\xfd\xdav\xb0\x19<_\x90\x01\x08\xac\xbfP\xec6\xdcb\xda\xc4\xfb/\xbaq/\xa4\xca_d\xfd\xba\xf4\xd5\x8e\xc9\xc9\xf2\xe9\xf8\xd8c9\xe7\x9b\xaa_\x05V\xbd\xda\xe9(\x1f[\xf4j4\xd5\x97\x1b\xfa7\x04\xd6~\xd1\x1eW+\x96{T\xf7a\xfb|]|J/\x16\xce[\xde\t\xac\x0f\x8e\xed\xa4\xfd\xda\xb6\xdf\x0c^,\xc8\x00\x04\xd6_hw_\xa6)o\x1ce\xce\x92\xc7\x17n\xf3\xd1]\xacjj\xb89xi5\xe7\xcd\xea\xd1\x17\x81Ut\xeb\x83\xa4~\xb5\x1fvW\xda\xddq\xd5\xeb\xbd\x88o\x08\xac\xfd\xa2=\xaeV.\x16s\xd9\xcf\xfc*\xc1^-\x9cw\'\xe3/\x07\xf8\xe3\x81\xdf\x1b\xdffm;\xdc\x1f}\xb6 \x03\x08\x1cX\xc5x\xcej\xfa\xf6\\|0\xe3\x9f\xfd\xff\xcb\xbe\x05\xb9n\x0f7\xae\xb6\xee_Z\xae3\xe3\x08\xe7\xe3\x89\xe1\x85\xb1yr\xae\xb1p\x1b\xbe\xcf\xd6\xe3,WmY\xc5z\xba\xca]`\xad\xe7`?\x05\x07C\xccR\xf7^`\xddV;\x18_\x04V\xea6\xbb\x8cy\x8b\xde\x1c\xe7T\xcb\xc3\xae\xa3)\x1d\xc6<>\xb5\x9d\xbby\xc7s|\xa1\x98\xc3\xaf\x9d\xbe\xf0\x8f\xe7\xf9`\xd1\x1e\xcf\xc7r1\x8f\xa3\\-\xa5\x17\x0b\xe71\t\xcby\\N\xf5\xb2\xd6\xf1d.\xd6\x8e\xc3w\xde\xf7\x8a\x8b\xc3\x81\xb7\x83\x0f\xd39\xae\xa3\xe5n\xca\xa7?_\rS.\x02k\xb5\x19\xbcZ\x90\xc3\xeb\xe3\xc0/6\x8c\x1f\x156\xb0\x9a\xf9\xd4\xd0\xf8\xf5y\x14Xy\x98\xb4\xdf\xe0\xe7\x97\xe6v\xa02\x8f0\xefc\xf4/\xb4\x9b\x1a\xeb\xc0\xaa\x1f5G\xc5c\x85\xcf\x7f>2`\xfdM\xbf\x9f\x83\xfd\x14\xec\x86X\xbe\xbb\xb8\xfb\xb2\rk=M_\x05\xd6\xc1>\xe0\xb8E7\x9b)(\x9bf\xfd%\xb0\x9d\xd2~\xcc\xf5\xb3\xb9k\x96SU\xcdmNS\xa9\'\xf3|\xb0h

In [5]:
dwi_w.get_changed_df()
topupdn_w.get_changed_df()
all_dwi_visqc = pd.concat([dwi_w.get_changed_df(), topupdn_w.get_changed_df()], axis=1)
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))

In [5]:
topupdn_w.get_changed_df()

,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc
0,0,0,0,True,0,0,True
1,1,1,0,True,1,0,True
2,2,2,0,True,2,0,True
3,3,3,0,True,3,0,True
4,4,4,0,True,4,0,True
5,5,5,0,True,5,0,True
6,6,6,0,True,6,0,True
7,-9999,7,0,False,-9999,-9999,False
8,-9999,-9999,-9999,False,-9999,-9999,False
9,-9999,-9999,-9999,False,-9999,-9999,False


In [7]:
dwi_w.get_changed_df()

,bvals,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec
0,0,0,0,True,0.000,0.000,0.000
1,2000,0,1,True,1.000,0.000,0.000
2,2000,1,0,True,0.000,1.000,0.000
3,2000,2,0,True,0.000,0.000,1.000
4,2000,3,0,True,-0.625,-0.777,0.076
5,2000,4,0,True,-0.745,0.623,0.237
6,2000,5,1,True,0.025,-0.095,0.995
7,2000,6,0,True,-0.853,-0.004,-0.522
8,2000,7,1,True,-0.155,0.904,-0.399
9,2000,8,1,True,-0.025,-0.716,-0.698
